In [31]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
library(reshape2)
library(rstatix)
library(ggpubr)


# Get DEG Number

In [32]:
DEG_res<-read.csv("/home//jupyter/BRI_Figures_Final_V1/Figure4/01_DEG_All_Samples/Deseq2_Result_Y1D0.csv")

In [33]:
DEG_res_signi<-DEG_res %>% filter(padj<0.05,abs(log2FoldChange)>0.1)

In [34]:
DEG_counts<-as.data.frame(table(DEG_res_signi$contrast,DEG_res_signi$celltype))

In [35]:
colnames(DEG_counts)<-c("factor","AIFI_L3","DEG_Counts")

# Get Freq

In [36]:
BRI_Freq_Data<-read.csv("/home/jupyter/BRI_Figures_Final_V1/Dataset/diha_AIFI_L3_frequencies_2024-05-05.csv")

In [37]:
BRI_Freq_Data<-BRI_Freq_Data %>% group_by(sample.sampleKitGuid) %>%
  dplyr::mutate(percentage = (AIFI_L3_count / sum(AIFI_L3_count)) * 100) %>% ungroup()

In [38]:
BRI_Freq_Data_subset<-BRI_Freq_Data %>% filter(sample.visitName=="Flu Year 1 Day 0")

### CMV


In [39]:
#compute freq changes
freq_changes<-BRI_Freq_Data_subset %>%
  select(percentage, AIFI_L3, cohort.cohortGuid, subject.biologicalSex, subject.cmv) %>%
  group_by(AIFI_L3, subject.cmv) %>%
  summarise(mean_AIFI_L3_clr = median(percentage, na.rm = TRUE)) %>%
  spread(subject.cmv, mean_AIFI_L3_clr) %>%
  mutate(log2fc = log2(`Positive` / `Negative`)) %>%
  mutate(delta_change = `Positive` - `Negative`) 
#compute freq changes p values

freq_changes_STAT<-BRI_Freq_Data_subset %>%
  select(AIFI_L3_clr,AIFI_L3, cohort.cohortGuid,subject.biologicalSex,subject.cmv) %>%
  group_by(AIFI_L3)  %>%
  wilcox_test(AIFI_L3_clr ~ subject.cmv) %>%
  adjust_pvalue(method='BH') %>%
  add_significance() %>% as.data.frame()

df<-left_join(freq_changes_STAT,freq_changes)

`summarise()` has grouped output by 'AIFI_L3'. You can override using the
`.groups` argument.
Joining with `by = join_by(AIFI_L3)`


In [40]:
df<-left_join(df,DEG_counts %>% filter(factor=="CMV"))

Joining with `by = join_by(AIFI_L3)`


In [41]:
df<-df%>%
  mutate(adjP_degs_combined_value = if_else(DEG_Counts != 0,
                                            -log10(p.adj)  * DEG_Counts * 0.1,
                                            -log10(p.adj)  * 0.1))

In [19]:
write.csv(df,'FreqChanges_and_DEG_Y1D0.csv')

# Upload Files

In [7]:
input_uuid=pull(read.csv("/home/jupyter/BRI_Figures_Final_V1/Dataset/scRNA_BRI_counts_summary_uuid.csv")['id'])


In [9]:
study_space_uuid <- 'de025812-5e73-4b3c-9c3b-6d0eac412f2a'
date_today <- Sys.Date()
working_dir <- getwd()
notebook_name <- basename("BRI_Figures_Final_V1/Figure2/03_Frequency_Changes/01_FreqChange_and_DEG.ipynb")  

title <- sprintf("Cert-Pro_IHA_Figures_Files_%s_from_%s/%s", date_today, working_dir, notebook_name)
title <- str_replace_all(title, c("/" = "-", "\\." = "_"))

title

[1] "Cert-Pro_IHA_Figures_Files_2024-06-10_from_-home-jupyter-BRI_Figures_Final_V1-Figure2-03_Frequency_Changes-01_FreqChange_and_DEG_ipynb"

In [13]:
uploadFiles(
  list('/home/jupyter/BRI_Figures_Final_V1/Figure2/03_Frequency_Changes/FreqChanges_and_DEG.csv'),
  studySpaceId = 'de025812-5e73-4b3c-9c3b-6d0eac412f2a',
  title = title, fileTypes = list('csv'),destination=title,
  inputFileIds = as.list(c(input_uuid))
)

[1] "Cannot determine the current notebook."
[1] "1) /home/jupyter/BRI_Figures_Final_V1/Figure2/03_Frequency_Changes/01_FreqChange_and_DEG.ipynb"
[1] "2) /home/jupyter/BRI_Figures_Final_V1/Figure2/02_DEG/02B_DEG_analysis.ipynb"
[1] "3) /home/jupyter/Additional_Analysis/CMV_Tao/02B_BR1_CMVpos_DEG_analysis.ipynb"


Please select (1-3)  1


You are trying to upload the following files:  /home/jupyter/BRI_Figures_Final_V1/Figure2/03_Frequency_Changes/FreqChanges_and_DEG.csv



(y/n) y


$files
$files[[1]]
[1] "/home/jupyter/BRI_Figures_Final_V1/Figure2/03_Frequency_Changes/FreqChanges_and_DEG.csv"


$traceId
[1] "d4dca952-2324-49a8-85aa-5a2a11d89555"